In [4]:
import requests
import pandas as pd
import numpy as np
#import eurostat
import json
import os
import datetime as dt

In [6]:
import os
print("Working directory:", os.getcwd())

Working directory: /Users/lorenz/Library/CloudStorage/OneDrive-UniversitàdegliStudidiMilano/PhD/Progetto/GaR/src


In [ ]:
os.chdir('\\Users\\lorenz\\Library\\CloudStorage\\OneDrive-UniversitàdegliStudidiMilano\\PhD\\Progetto\\GaR\\src')
path = 'data'

In [7]:
# IMPORT INDUSTRIAL PRODUCTION (IP) DATA 

ip = pd.read_csv("ip_by_sector.csv")

ip = ip[['nace_r2', 'geo', 'TIME_PERIOD', 'OBS_VALUE', 'unit']]

ip['Time'] = ip['TIME_PERIOD']

#ip['Time'] = pd.to_datetime(ip['Time'], format="%Y-%m")    

#ip['Time'] = ip['Time'].dt.strftime('%Y/%m')

ip.set_index('Time', inplace=True)

In [52]:
# IMPORT GAS PRICES DATA 

gasp = pd.read_csv(os.path.join(path, "gas_data.csv"))

gasp[['Prices']] = gasp[['LSG Natural Gas TTF NL 1st Fut. Day - SETT. PRICE']]

gasp = gasp[['Time','Prices']]

gasp['Time'] = pd.to_datetime(gasp['Time'], format="%m/%d/%y")

month_gasp = gasp.groupby(gasp['Time'].dt.to_period('M')).mean() # group data and make monthly average to match IP data

month_gasp = month_gasp.drop(columns=['Time'])

month_gasp.loc['2004-01':'2023-12']

,Prices
Time,
2004-01,11.360000
2004-02,10.797500
2004-03,10.900435
2004-04,10.974545
2004-05,11.221905
...,...
2023-08,33.523043
2023-09,36.527619
2023-10,43.264545


In [53]:
df = ip.drop(columns=['TIME_PERIOD'])

df['geo'] = df['geo'].astype('category')

df['nace_r2'] = df['nace_r2'].astype('category')

df['unit'] = df['unit'].astype('category')

df_ue = df[df['geo'] == 'EU27_2020']

df_ue_c = df_ue[df_ue['nace_r2'] == 'C']

df_ue_c21 = df_ue_c[df_ue_c['unit'] == 'I15']

df_ue_c21['GAS_PRICE'] = month_gasp['Prices']

C:\Users\loren\AppData\Local\Temp\ipykernel_44280\3386429221.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_ue_c21['GAS_PRICE'] = month_gasp['Prices']


In [54]:
df_ue_c21.to_csv(os.path.join(path, "sample_df.csv"))